In [ ]:
import numpy as np
import pandas as pd

from dotenv import load_dotenv
import os

import unicodedata
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.metrics import precision_recall_curve, auc, precision_score, recall_score, f1_score, fbeta_score, roc_curve, average_precision_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
import logging
import json
import re
import string 
import joblib
import warnings
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import mlflow
import mlflow.sklearn
from sklearn.pipeline import Pipeline
from mlflow.models.signature import infer_signature

warnings.filterwarnings("ignore")

In [ ]:
# Configuración del logging
logging.basicConfig(
    filename="errores_entrenamiento.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s"
)


In [ ]:
# Load environment variables
try:
    load_dotenv()
    ruta_cst_twcs = os.getenv("customer_support_twitter_twcs")
    logging.info("Environment variables loaded successfully.")
except Exception as e:
    logging.error(f"Error loading environment variables: {e}")
    raise e

In [ ]:
# load data
try:
    data_cst_twcs = pd.read_csv(ruta_cst_twcs)
    print(data_cst_twcs.shape)
    logging.info("Data loaded successfully.")
except FileNotFoundError as e:
    logging.error(f"File not found: {ruta_cst_twcs}")
    raise e

In [ ]:
# transform the 'inbound' column to int
try:
    data_cst_twcs['inbound'] = data_cst_twcs['inbound'].astype('int')
    logging.info("Data transformed successfully.")
except Exception as e:
    logging.error(f"Error transforming data: {e}")
    raise e

In [ ]:
# load stopwords
try:
    nltk.download('punkt')
    #nltk.download('wordnet')
    nltk.download('stopwords')
    english_stopwords = stopwords.words('english')
except Exception as e:
    logging.error(f"Error loading stopwords: {e}")
    raise e

In [ ]:
# split the data into train, validation and test sets
# stratified split to maintain the same proportion of classes in each set
try:
    X_train, X_test, y_train, y_test = train_test_split(data_cst_twcs['text'], data_cst_twcs['inbound'], test_size=0.3, stratify=data_cst_twcs['inbound'], random_state=42)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train, random_state=42)
    print('X_train: ', X_train.shape)
    print('X_valid: ', X_valid.shape)
    print('X_test: ', X_test.shape)
    print('y_train: ', y_train.shape)
    print('y_valid: ', y_valid.shape)
    print('y_test: ', y_test.shape)
    logging.info("Data split into train, validation and test sets successfully.")
except Exception as e:
    logging.error(f"Error splitting data: {e}")
    raise e

In [ ]:
# Vectorizar el texto con TF-IDF
#vectorizer = TfidfVectorizer(stop_words=english_stopwords, max_features=100, lowercase=True, token_pattern=r'\b\w+\b')

# Fit the vectorizer on the training data and transform the train, validation and test sets
#X_train = vectorizer.fit_transform(X_train)
#X_valid = vectorizer.transform(X_valid)
#X_test = vectorizer.transform(X_test)

In [ ]:
# Set the experiment name
try:
    mlflow.create_experiment("experimento_catboost")
    print("Experimento creado")
    logging.info("Experiment created successfully.")
except:
    mlflow.set_experiment("experimento_catboost")
    print("Experimento ya existe")
    logging.info("Experiment already exists, set to existing experiment.")

In [ ]:
try:
    print("Iniciando el experimento...")
    with mlflow.start_run():
        # Define y entrena el pipeline
        catboost_params = {
            "iterations": 500,
            "depth": 6,
            "learning_rate": 0.1,
            "verbose": 100
        }

        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(stop_words=english_stopwords, max_features=100, lowercase=True, token_pattern=r'\b\w+\b')),
            ('catboost', CatBoostClassifier(**catboost_params))
        ])
        pipeline.fit(X_valid, y_valid)

        # Inferir signature para input/output
        signature = infer_signature(X_valid, y_valid)

        # Registra el modelo con un ejemplo de entrada
        input_example = np.array(X_train[:1])  # Toma una muestra como ejemplo de entrada
        mlflow.sklearn.log_model(pipeline, "modelo_catboost", input_example=input_example, signature=signature, registered_model_name="modelo_catboost_prueba")

        # Registra las métricas
        accuracy = pipeline.score(X_test, y_test)
        mlflow.log_metric("accuracy", accuracy)

        # Registra los hiperparámetros del modelo
        mlflow.log_params(catboost_params)

        print(f"Modelo registrado con precisión: {accuracy}")
        logging.info(f"Modelo registrado con precisión: {accuracy}")
except Exception as e:
    logging.error(f"Error during MLflow run: {e}")
    raise e

# MLOps - Proceso de Reentrenamiento y Comparación de Modelos

Este notebook implementa un proceso MLOps para:
1. Entrenar un nuevo modelo (challenger)
2. Recuperar el modelo campeón actual de MLflow (champion)
3. Comparar ambos modelos usando métricas definidas
4. Registrar como nuevo campeón el que tenga mejor desempeño
5. Documentar toda la experimentación en MLflow

In [1]:
import numpy as np
import pandas as pd
    
from dotenv import load_dotenv
import os
    
import unicodedata
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.metrics import precision_recall_curve, auc, precision_score, recall_score, f1_score, fbeta_score, roc_curve, average_precision_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
import logging
import json
import re
import string 
import joblib
import warnings
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import mlflow
import mlflow.sklearn
from sklearn.pipeline import Pipeline
from mlflow.models.signature import infer_signature
from datetime import datetime
    
warnings.filterwarnings('ignore')

In [2]:
# Configuración del logging
logging.basicConfig(
    filename=f'reentrenamiento_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log',
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
    )
    

In [3]:
# Función para registrar mensajes tanto en log como en consola
def log_info(message):
    print(message)
    logging.info(message)
        
def log_error(message):
    print(f'ERROR: {message}')
    logging.error(message)

In [4]:
# Cargar variables de entorno
try:
    load_dotenv()
    ruta_cst_twcs = os.getenv('customer_support_twitter_twcs')
    # Configurar nombre del modelo campeón y experimento
    CHAMPION_MODEL_NAME = os.getenv('CHAMPION_MODEL_NAME', 'modelo_catboost_champion')
    EXPERIMENT_NAME = os.getenv('EXPERIMENT_NAME', 'experimento_catboost')
    # Configurar umbral para considerar un modelo mejor
    IMPROVEMENT_THRESHOLD = float(os.getenv('IMPROVEMENT_THRESHOLD', '0.01'))  # 1% de mejora por defecto
    # Métrica principal para comparación
    PRIMARY_METRIC = os.getenv('PRIMARY_METRIC', 'f1')
    print('Variables de entorno cargadas correctamente.')
    log_info('Variables de entorno cargadas correctamente.')
except Exception as e:
    print(f'Error al cargar variables de entorno: {e}')
    log_error(f'Error al cargar variables de entorno: {e}')
    raise e


Variables de entorno cargadas correctamente.
Variables de entorno cargadas correctamente.


In [5]:
# Cargar datos
try:
    data_cst_twcs = pd.read_csv(ruta_cst_twcs)
    print(f'Dimensiones del dataset: {data_cst_twcs.shape}')
    log_info('Datos cargados correctamente.')
except FileNotFoundError as e:
    log_error(f'Archivo no encontrado: {ruta_cst_twcs}')
    raise e


Dimensiones del dataset: (2811774, 7)
Datos cargados correctamente.


In [6]:
# Transformar la columna 'inbound' a int
try:
    data_cst_twcs['inbound'] = data_cst_twcs['inbound'].astype('int')
    log_info('Datos transformados correctamente.')
except Exception as e:
    log_error(f'Error al transformar datos: {e}')
    raise e

Datos transformados correctamente.


In [7]:
# Cargar recursos NLP
try:
    nltk.download('punkt')
    nltk.download('stopwords')
    english_stopwords = stopwords.words('english')
    sentence_tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
    log_info('Recursos NLP cargados correctamente.')
except Exception as e:
    log_error(f'Error al cargar stopwords: {e}')
    raise e

Recursos NLP cargados correctamente.


[nltk_data] Downloading package punkt to /home/alejo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/alejo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
data_cst_twcs['text']

0          @115712 I understand. I would like to assist y...
1              @sprintcare and how do you propose we do that
2          @sprintcare I have sent several private messag...
3          @115712 Please send us a Private Message so th...
4                                         @sprintcare I did.
                                 ...                        
2811769    @823869 Hey, we'd be happy to look into this f...
2811770    @115714 wtf!? I’ve been having really shitty s...
2811771    @143549 @sprintcare You have to go to https://...
2811772    @823870 Sounds delicious, Sarah! 😋 https://t.c...
2811773    @AldiUK  warm sloe gin mince pies with ice cre...
Name: text, Length: 2811774, dtype: object

In [ ]:
# Aplicar la tokenización de oraciones a cada fila
#data_cst_twcs['tokenized_sentences'] = data_cst_twcs['text'].apply(sentence_tokenizer.tokenize)
#data_cst_twcs['tokenized_sentences']

0          [@115712 I understand., I would like to assist...
1            [@sprintcare and how do you propose we do that]
2          [@sprintcare I have sent several private messa...
3          [@115712 Please send us a Private Message so t...
4                                       [@sprintcare I did.]
                                 ...                        
2811769    [@823869 Hey, we'd be happy to look into this ...
2811770    [@115714 wtf!?, I’ve been having really shitty...
2811771    [@143549 @sprintcare You have to go to https:/...
2811772    [@823870 Sounds delicious, Sarah!, 😋 https://t...
2811773    [@AldiUK  warm sloe gin mince pies with ice cr...
Name: tokenized_sentences, Length: 2811774, dtype: object

In [14]:
# Dividir los datos en conjuntos de entrenamiento, validación y prueba
# División estratificada para mantener la misma proporción de clases en cada conjunto
try:
    X_train, X_test, y_train, y_test = train_test_split(data_cst_twcs['text'], data_cst_twcs['inbound'], 
                                                        test_size=0.3, stratify=data_cst_twcs['inbound'], 
                                                        random_state=42)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, 
                                                        test_size=0.3, stratify=y_train, 
                                                        random_state=42)
    print('X_train: ', X_train.shape)
    print('X_valid: ', X_valid.shape)
    print('X_test: ', X_test.shape)
    print('y_train: ', y_train.shape)
    print('y_valid: ', y_valid.shape)
    print('y_test: ', y_test.shape)
    log_info('Datos divididos en conjuntos de entrenamiento, validación y prueba correctamente.')
except Exception as e:
    log_error(f'Error al dividir los datos: {e}')
    raise e

X_train:  (1377768,)
X_valid:  (590473,)
X_test:  (843533,)
y_train:  (1377768,)
y_valid:  (590473,)
y_test:  (843533,)
Datos divididos en conjuntos de entrenamiento, validación y prueba correctamente.


In [12]:
# Set the experiment name
try:
    mlflow.create_experiment("experimento_catboost")
    print("Experimento creado")
    logging.info("Experiment created successfully.")
except:
    mlflow.set_experiment("experimento_catboost")
    print("Experimento ya existe")
    logging.info("Experiment already exists, set to existing experiment.")

Experimento ya existe


In [15]:
try:
    print("Iniciando el experimento...")
    with mlflow.start_run():
        # Define y entrena el pipeline
        catboost_params = {
            "iterations": 500,
            "depth": 6,
            "learning_rate": 0.1,
            "verbose": 100
        }

        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(stop_words=english_stopwords, max_features=100, lowercase=True, token_pattern=r'\b\w+\b')),
            ('catboost', CatBoostClassifier(**catboost_params))
        ])
        pipeline.fit(X_train, y_train)

        # Inferir signature para input/output
        signature = infer_signature(X_train, y_train)

        # Registra el modelo con un ejemplo de entrada
        input_example = np.array(X_train[:1])  # Toma una muestra como ejemplo de entrada
        mlflow.sklearn.log_model(pipeline, "modelo_catboost", input_example=input_example, signature=signature, registered_model_name="modelo_catboost_prueba")

        # Registra las métricas
        accuracy = pipeline.score(X_test, y_test)
        mlflow.log_metric("accuracy", accuracy)

        # Registra los hiperparámetros del modelo
        mlflow.log_params(catboost_params)

        print(f"Modelo registrado con precisión: {accuracy}")
        logging.info(f"Modelo registrado con precisión: {accuracy}")
except Exception as e:
    logging.error(f"Error during MLflow run: {e}")
    raise e

Iniciando el experimento...
0:	learn: 0.6316539	total: 2.51s	remaining: 20m 52s
100:	learn: 0.3638261	total: 26.8s	remaining: 1m 45s
200:	learn: 0.3455131	total: 51s	remaining: 1m 15s
300:	learn: 0.3375544	total: 1m 14s	remaining: 49s
400:	learn: 0.3330456	total: 1m 37s	remaining: 24.2s
499:	learn: 0.3297846	total: 2m 1s	remaining: 0us


Registered model 'modelo_catboost_prueba' already exists. Creating a new version of this model...
Created version '5' of model 'modelo_catboost_prueba'.


Modelo registrado con precisión: 0.8513466574514571


## Implementación del Flujo MLOps
    
A continuación, implementamos las funciones para:
1. Recuperar el modelo campeón actual
2. Entrenar un nuevo modelo
3. Evaluar y comparar ambos modelos
4. Promover el mejor modelo como nuevo campeón

In [9]:
def evaluate_model(model, X, y, model_name='modelo'):
    '''Evalúa un modelo y devuelve un diccionario con múltiples métricas.'''
    try:
        # Predicciones
        y_pred = model.predict(X)
        y_pred_proba = model.predict_proba(X)[:, 1] if hasattr(model, 'predict_proba') else None
            
        # Métricas básicas
        metrics = {
            'accuracy': accuracy_score(y, y_pred),
            'precision': precision_score(y, y_pred),
            'recall': recall_score(y, y_pred),
            'f1': f1_score(y, y_pred),
        }
            
        # Métricas avanzadas si hay probabilidades
        if y_pred_proba is not None:
            metrics['roc_auc'] = roc_auc_score(y, y_pred_proba)
            metrics['avg_precision'] = average_precision_score(y, y_pred_proba)
            
        # Reporte de clasificación
        report = classification_report(y, y_pred, output_dict=True)
            
        log_info(f'Evaluación de {model_name}:')
        for metric, value in metrics.items():
            log_info(f'- {metric}: {value:.4f}')
                
        return metrics, report
    except Exception as e:
        log_error(f'Error al evaluar el modelo {model_name}: {e}')
        raise e


In [10]:
def plot_confusion_matrix(model, X, y, title='Matriz de Confusión'):
    '''Grafica la matriz de confusión para un modelo.'''
    try:
        fig, ax = plt.subplots(figsize=(8, 6))
        ConfusionMatrixDisplay.from_estimator(model, X, y, ax=ax)
        plt.title(title)
        plt.tight_layout()
        return fig
    except Exception as e:
        log_error(f'Error al graficar matriz de confusión: {e}')
        return None


In [11]:
def plot_roc_curve(model, X, y, model_name='Modelo'):
    '''Grafica la curva ROC para un modelo.'''
    try:
        if not hasattr(model, 'predict_proba'):
            log_info(f'El modelo {model_name} no soporta predict_proba, no se puede graficar ROC.')
            return None
                
        y_pred_proba = model.predict_proba(X)[:, 1]
        fpr, tpr, _ = roc_curve(y, y_pred_proba)
        roc_auc = auc(fpr, tpr)
            
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.4f})')
        ax.plot([0, 1], [0, 1], 'k--')
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title('Curva ROC')
        ax.legend(loc='lower right')
        return fig
    except Exception as e:
        log_error(f'Error al graficar curva ROC: {e}')
        return None


In [ ]:
def get_champion_model():
    '''Recupera el modelo campeón actual desde MLflow.'''
    try:
        client = mlflow.tracking.MlflowClient()
            
        # Buscar la última versión del modelo campeón
        try:
            latest_version = client.get_latest_versions(CHAMPION_MODEL_NAME, stages=['Production'])
            if not latest_version:
                log_info(f'No se encontró un modelo {CHAMPION_MODEL_NAME} en producción. Buscando en staging...')
                latest_version = client.get_latest_versions(CHAMPION_MODEL_NAME, stages=['Staging'])
                    
            if not latest_version:
                log_info(f'No se encontró un modelo {CHAMPION_MODEL_NAME} en staging. Buscando la versión más reciente...')
                latest_version = client.get_latest_versions(CHAMPION_MODEL_NAME)
                    
            if latest_version:
                model_uri = f'models:/{CHAMPION_MODEL_NAME}/{latest_version[0].version}'
                champion_model = mlflow.sklearn.load_model(model_uri)
                log_info(f'Modelo campeón cargado: {CHAMPION_MODEL_NAME} version {latest_version[0].version}')
                return champion_model, latest_version[0].run_id
            else:
                log_info(f'No se encontró ningún modelo registrado con el nombre {CHAMPION_MODEL_NAME}')
                return None, None
        except Exception as e:
            log_error(f'No se pudo obtener la última versión del modelo: {e}')
            return None, None
        
    except Exception as e:
        log_error(f'Error al recuperar el modelo campeón: {e}')
        return None, None


In [13]:
def register_challenger_model(model, metrics, X_train, y_train, is_champion=False):
    '''Registra un modelo desafiante en MLflow.'''
    try:
        # Inferir firma para input/output
        signature = infer_signature(X_train, y_train)
            
        # Registrar el modelo con un ejemplo de entrada
        input_example = np.array(X_train[:1])
            
        # Nombre del modelo y etapa
        model_name = CHAMPION_MODEL_NAME if is_champion else f'{CHAMPION_MODEL_NAME}_challenger'
        stage = 'Production' if is_champion else 'Staging'
            
        # Registrar modelo
        mlflow.sklearn.log_model(
            model, 
            'model', 
            input_example=input_example, 
            signature=signature, 
            registered_model_name=model_name
        )
            
        # Registrar métricas
        for metric_name, metric_value in metrics.items():
            mlflow.log_metric(metric_name, metric_value)
                
        # Si es el campeón, mover a producción
        if is_champion:
            client = mlflow.tracking.MlflowClient()
            latest_version = client.get_latest_versions(model_name)[0].version
            client.transition_model_version_stage(
                name=model_name,
                version=latest_version,
                stage=stage
            )
            log_info(f'Modelo {model_name} v{latest_version} promocionado a {stage}')
            
        return True
    except Exception as e:
        log_error(f'Error al registrar el modelo: {e}')
        return False


In [15]:
def train_challenger_model(X_train, y_train, X_valid=None, y_valid=None, hyperparams=None):
    '''Entrena un nuevo modelo desafiante.'''
    try:
        # Parámetros por defecto si no se especifican
        if hyperparams is None:
            hyperparams = {
                'iterations': 500,
                'depth': 6,
                'learning_rate': 0.1,
                'verbose': 100,
                'max_features': 100
            }
            
        # Extraer parámetros específicos de vectorizador y modelo
        max_features = hyperparams.pop('max_features', 100)
            
        log_info(f'Entrenando modelo desafiante con parámetros: {hyperparams}')
            
        # Crear pipeline
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(stop_words=english_stopwords, max_features=max_features, lowercase=True, token_pattern=r'\b\w+\b')),              
            ('catboost', CatBoostClassifier(**hyperparams))
        ])
            
        # Entrenar modelo
        if X_valid is not None and y_valid is not None:
            # Usar conjunto de validación para early stopping
            pipeline.fit(X_train, y_train, catboost__eval_set=[(X_valid, y_valid)])
        else:
            pipeline.fit(X_train, y_train)
            
        log_info('Modelo desafiante entrenado correctamente')
        return pipeline
    except Exception as e:
        log_error(f'Error al entrenar modelo desafiante: {e}')
        raise e


In [16]:
def optimize_hyperparameters(X_train, y_train, X_valid, y_valid, max_evals=10):
    '''Optimiza hiperparámetros usando Hyperopt.'''
    try:
        log_info('Iniciando optimización de hiperparámetros...')
            
        # Definir espacio de búsqueda
        space = {
            'max_features': hp.choice('max_features', [50, 100, 200, 300]),
            'iterations': hp.choice('iterations', [300, 500, 700]),
            'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),
            'depth': hp.choice('depth', [4, 6, 8, 10]),
            'l2_leaf_reg': hp.loguniform('l2_leaf_reg', np.log(1), np.log(10)),
            'verbose': 0
        }
            
        # Función objetivo para minimizar
        def objective(params):
            # Extraer max_features para TfidfVectorizer
            max_features = params.pop('max_features', 100)
                
            # Crear y entrenar pipeline
            pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=english_stopwords, max_features=max_features, 
                                            lowercase=True, token_pattern=r'\b\w+\b')),
                ('catboost', CatBoostClassifier(**params))
            ])
                
            try:
                pipeline.fit(X_train, y_train, catboost__eval_set=[(X_valid, y_valid)])
                    
                # Evaluar en conjunto de validación
                y_pred = pipeline.predict(X_valid)
                f1 = f1_score(y_valid, y_pred)
                    
                # Reintegrar max_features al diccionario de parámetros
                params['max_features'] = max_features
                    
                return {'loss': -f1, 'status': STATUS_OK, 'params': params}
            except Exception as e:
                log_error(f'Error en evaluación de hiperparámetros: {e}')
                return {'loss': 0, 'status': STATUS_OK, 'params': params}
            
        # Ejecutar optimización
        trials = Trials()
        best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
            
        # Obtener mejores parámetros
        best_params = trials.results[np.argmin([r['loss'] for r in trials.results])]['params']
            
        log_info(f'Mejores hiperparámetros encontrados: {best_params}')
        return best_params
    except Exception as e:
        log_error(f'Error en optimización de hiperparámetros: {e}')
        # Devolver parámetros por defecto en caso de error
        return {
            'iterations': 500,
            'depth': 6,
            'learning_rate': 0.1,
            'verbose': 100,
            'max_features': 100
        }


In [17]:
def compare_models(champion_metrics, challenger_metrics, primary_metric=PRIMARY_METRIC, threshold=IMPROVEMENT_THRESHOLD):
    '''Compara los modelos y determina si el desafiante debe convertirse en el nuevo campeón.'''
    try:
        if champion_metrics is None:
            log_info('No hay modelo campeón para comparar. El desafiante se convierte en campeón por defecto.')
            return True, {}, {'champion': None, 'challenger': challenger_metrics[primary_metric]}
            
        # Comparar métricas primarias
        champion_score = champion_metrics[primary_metric]
        challenger_score = challenger_metrics[primary_metric]
            
        improvement = challenger_score - champion_score
        percent_improvement = (improvement / champion_score) * 100 if champion_score > 0 else float('inf')
            
        comparison = {
            'champion': champion_score,
            'challenger': challenger_score,
            'absolute_diff': improvement,
            'percent_diff': percent_improvement
        }
            
        # Comparar todas las métricas disponibles
        all_metrics = {}
        for metric in set(champion_metrics.keys()).union(challenger_metrics.keys()):
            if metric in champion_metrics and metric in challenger_metrics:
                champion_val = champion_metrics[metric]
                challenger_val = challenger_metrics[metric]
                diff = challenger_val - champion_val
                perc_diff = (diff / champion_val) * 100 if champion_val > 0 else float('inf')
                    
                all_metrics[metric] = {
                    'champion': champion_val,
                    'challenger': challenger_val,
                    'absolute_diff': diff,
                    'percent_diff': perc_diff
                }
            
        # Determinar si el desafiante es mejor
        is_better = improvement > threshold
            
        if is_better:
            log_info(f'El modelo desafiante es mejor en {primary_metric}: {challenger_score:.4f} vs {champion_score:.4f} ')
            log_info(f'Mejora absoluta: {improvement:.4f}, Mejora porcentual: {percent_improvement:.2f}%')
        else:
            log_info(f'El modelo desafiante NO supera al campeón en {primary_metric}: {challenger_score:.4f} vs {champion_score:.4f}')
            log_info(f'Diferencia absoluta: {improvement:.4f}, Diferencia porcentual: {percent_improvement:.2f}%')
                
        return is_better, all_metrics, comparison
    except Exception as e:
        log_error(f'Error al comparar modelos: {e}')
        return False, {}, {}


## Flujo Principal MLOps
    
A continuación implementamos el flujo completo del proceso MLOps con los siguientes pasos:
1. Configurar MLflow
2. Obtener el modelo campeón actual
3. Optimizar hiperparámetros para el modelo desafiante
4. Entrenar el modelo desafiante
5. Evaluar ambos modelos
6. Comparar modelos y seleccionar el mejor
7. Registrar el nuevo campeón si corresponde


In [19]:
# Configurar MLflow
try:
    # Crear experimento si no existe
    try:
        mlflow.create_experiment(EXPERIMENT_NAME)
        log_info(f'Experimento "{EXPERIMENT_NAME}" creado correctamente')
    except:
        mlflow.set_experiment(EXPERIMENT_NAME)
        log_info(f'Experimento "{EXPERIMENT_NAME}" ya existe, usando experimento existente')
except Exception as e:
    log_error(f'Error al configurar MLflow: {e}')
    raise e


Experimento "experimento_catboost" ya existe, usando experimento existente


In [20]:
# Implementar el flujo MLOps completo
try:
    with mlflow.start_run(run_name=f'reentrenamiento_{datetime.now().strftime('%Y%m%d_%H%M')}') as run:
        log_info('=== INICIANDO PROCESO DE REENTRENAMIENTO MLOPS ===')
            
        # 1. Obtener modelo campeón
        log_info('1. Obteniendo modelo campeón...')
        champion_model, champion_run_id = get_champion_model()
            
        if champion_model is not None:
            log_info('Modelo campeón cargado correctamente')
            # Registrar información del modelo campeón
            mlflow.set_tag('champion_run_id', champion_run_id)
        else:
            log_info('No se encontró modelo campeón existente')
            
        # 2. Optimizar hiperparámetros (si hay suficientes datos)
        if len(X_train) > 1000:  # Solo optimizar si hay suficientes datos
            log_info('2. Optimizando hiperparámetros para modelo desafiante...')
            best_params = optimize_hyperparameters(X_train, y_train, X_valid, y_valid, max_evals=10)
        else:
            log_info('No hay suficientes datos para optimización de hiperparámetros, usando valores por defecto')
            best_params = {
                'iterations': 500,
                'depth': 6,
                'learning_rate': 0.1,
                'verbose': 100,
                'max_features': 100
            }
            
        # Registrar hiperparámetros
        for param_name, param_value in best_params.items():
            mlflow.log_param(param_name, param_value)
            
        # 3. Entrenar modelo desafiante
        log_info('3. Entrenando modelo desafiante...')
        challenger_model = train_challenger_model(X_train, y_train, X_valid, y_valid, best_params)
            
        # 4. Evaluar ambos modelos en conjunto de prueba
        log_info('4. Evaluando modelos en conjunto de prueba...')
            
        if champion_model is not None:
            champion_metrics, champion_report = evaluate_model(champion_model, X_test, y_test, 'Modelo Campeón')
            # Guardar matriz de confusión y curva ROC
            cm_fig_champion = plot_confusion_matrix(champion_model, X_test, y_test, 'Matriz de Confusión - Modelo Campeón')
            if cm_fig_champion:
                mlflow.log_figure(cm_fig_champion, 'confusion_matrix_champion.png')
                    
            roc_fig_champion = plot_roc_curve(champion_model, X_test, y_test, 'Modelo Campeón')
            if roc_fig_champion:
                mlflow.log_figure(roc_fig_champion, 'roc_curve_champion.png')
        else:
            champion_metrics = None
            champion_report = None
            
        challenger_metrics, challenger_report = evaluate_model(challenger_model, X_test, y_test, 'Modelo Desafiante')
            
        # Guardar matriz de confusión y curva ROC
        cm_fig_challenger = plot_confusion_matrix(challenger_model, X_test, y_test, 'Matriz de Confusión - Modelo Desafiante')
        if cm_fig_challenger:
            mlflow.log_figure(cm_fig_challenger, 'confusion_matrix_challenger.png')
                
        roc_fig_challenger = plot_roc_curve(challenger_model, X_test, y_test, 'Modelo Desafiante')
        if roc_fig_challenger:
            mlflow.log_figure(roc_fig_challenger, 'roc_curve_challenger.png')
            
        # 5. Comparar modelos
        log_info('5. Comparando modelos...')
        is_challenger_better, all_metrics_comparison, primary_comparison = compare_models(
            champion_metrics, challenger_metrics, PRIMARY_METRIC, IMPROVEMENT_THRESHOLD
        )
            
        # Guardar comparaciones
        mlflow.log_dict(all_metrics_comparison, 'metrics_comparison.json')
        mlflow.log_dict(primary_comparison, f'{PRIMARY_METRIC}_comparison.json')
            
        # 6. Registrar modelo ganador
        if is_challenger_better:
            log_info('6. El modelo desafiante es mejor. Promoviendo a nuevo campeón...')
            register_challenger_model(challenger_model, challenger_metrics, X_train, y_train, is_champion=True)
            mlflow.log_param('winner', 'challenger')
        else:
            log_info('6. El modelo campeón sigue siendo mejor. Registrando desafiante para referencia...')
            register_challenger_model(challenger_model, challenger_metrics, X_train, y_train, is_champion=False)
            mlflow.log_param('winner', 'champion')
            
        log_info('=== PROCESO MLOps COMPLETADO EXITOSAMENTE ===')
except Exception as e:
    log_error(f'Error en flujo MLOps: {e}')
    raise e


=== INICIANDO PROCESO DE REENTRENAMIENTO MLOPS ===
1. Obteniendo modelo campeón...
ERROR: No se pudo obtener la última versión del modelo: Registered Model with name=modelo_catboost_champion not found
No se encontró modelo campeón existente
2. Optimizando hiperparámetros para modelo desafiante...
Iniciando optimización de hiperparámetros...
ERROR: Error en evaluación de hiperparámetros: 0      
ERROR: Error en evaluación de hiperparámetros: 0                 
ERROR: Error en evaluación de hiperparámetros: 0                 
ERROR: Error en evaluación de hiperparámetros: 0                 
ERROR: Error en evaluación de hiperparámetros: 0                 
ERROR: Error en evaluación de hiperparámetros: 0                 
ERROR: Error en evaluación de hiperparámetros: 0                 
ERROR: Error en evaluación de hiperparámetros: 0                 
ERROR: Error en evaluación de hiperparámetros: 0                 
ERROR: Error en evaluación de hiperparámetros: 0                 
100%|███

KeyError: 0

## Visualizaciones de Resultados
    
A continuación, se muestran algunas visualizaciones adicionales para analizar y comparar los modelos.


In [ ]:
def plot_metrics_comparison(champion_metrics, challenger_metrics):
    '''Genera gráficos comparativos de métricas entre el modelo campeón y el desafiante.'''
    try:
        if champion_metrics is None:
            log_info('No hay métricas del modelo campeón para comparar.')
            return None
                
        # Encontrar métricas comunes
        common_metrics = set(champion_metrics.keys()).intersection(set(challenger_metrics.keys()))
            
        # Preparar datos para ploteo
        metrics_names = list(common_metrics)
        champion_values = [champion_metrics[metric] for metric in metrics_names]
        challenger_values = [challenger_metrics[metric] for metric in metrics_names]
            
        # Crear figura
        fig, ax = plt.subplots(figsize=(10, 6))
            
        # Configuración de barras
        x = np.arange(len(metrics_names))
        width = 0.35
            
        # Graficar barras
        ax.bar(x - width/2, champion_values, width, label='Modelo Campeón')
        ax.bar(x + width/2, challenger_values, width, label='Modelo Desafiante')
            
        # Añadir etiquetas y título
        ax.set_xlabel('Métricas')
        ax.set_ylabel('Valor')
        ax.set_title('Comparación de Métricas entre Modelos')
        ax.set_xticks(x)
        ax.set_xticklabels(metrics_names)
        ax.legend()
            
        # Añadir valores encima de cada barra
        for i, v in enumerate(champion_values):
            ax.text(i - width/2, v + 0.01, f'{v:.3f}', ha='center')
                
        for i, v in enumerate(challenger_values):
            ax.text(i + width/2, v + 0.01, f'{v:.3f}', ha='center')
                
        plt.tight_layout()
        return fig
    except Exception as e:
        log_error(f'Error al generar gráfico comparativo: {e}')
        return None


In [ ]:
# Visualizar comparación de métricas si hay dos modelos para comparar
if champion_metrics is not None and challenger_metrics is not None:
    comparison_fig = plot_metrics_comparison(champion_metrics, challenger_metrics)
    if comparison_fig:
        # Mostrar en notebook
        plt.show()
        # Guardar en MLflow
        with mlflow.start_run(run_id=run.info.run_id):
            mlflow.log_figure(comparison_fig, 'metrics_comparison.png')
else:
    print('No hay dos modelos para comparar métricas.')


## Automatización del Proceso MLOps
    
Para integrar este notebook en un flujo de trabajo automatizado MLOps, se puede exportar como script Python y programar su ejecución periódica usando herramientas como Apache Airflow, cron jobs, o plataformas CI/CD. A continuación se muestra cómo configurar los metadatos del experimento para facilitar el seguimiento.
    
Ejemplo de configuración para Airflow:

In [ ]:
# Este código no se ejecuta aquí, es solo un ejemplo de DAG para Airflow
'''
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta
import subprocess
    
default_args = {
    'owner': 'mlops',
    'depends_on_past': False,
    'email_on_failure': True,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}
    
def run_retraining():
    # Ejecutar script de reentrenamiento
    subprocess.run(['python', 'reentrenamiento_mlops.py'])
    
with DAG(
    'modelo_catboost_retraining',
    default_args=default_args,
    description='Reentrenamiento periódico del modelo de clasificación',
    schedule_interval=timedelta(days=7),  # Reentrenar semanalmente
    start_date=datetime(2025, 5, 1),
    catchup=False,
) as dag:
    retraining_task = PythonOperator(
        task_id='reentrenamiento_modelo',
        python_callable=run_retraining,
    )
'''


## Conclusiones
    
Este notebook implementa un flujo MLOps completo para:
    
1. **Entrenamiento automatizado**: Entrenamiento automatizado del modelo utilizando los últimos datos disponibles.
2. **Optimización de hiperparámetros**: Búsqueda automática de los mejores hiperparámetros para el modelo.
3. **Gestión de modelos**: Uso de MLflow para registrar y versionar modelos.
4. **Evaluación comparativa**: Comparación sistemática entre modelo campeón y desafiante.
5. **Promoción automática**: Promoción automática del mejor modelo a producción basado en métricas objetivas.
    
Este enfoque garantiza que solo los modelos que realmente mejoran el rendimiento sean promovidos a producción, manteniendo un historial completo de experimentos y decisiones.


In [ ]:
# Código para exportar este notebook como script Python (opcional)
try:
    !jupyter nbconvert --to python reentrenamiento_mlops.ipynb
    print('Notebook exportado como script Python: reentrenamiento_mlops.py')
except Exception as e:
    print(f'Error al exportar notebook: {e}')
